In [19]:
root='/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3'

In [ ]:
def print_batch(batch):
        for image, _, landmark in batch:
        image = (image - image.min()) / (image.max() - image.min())

        plt.figure(figsize=(6, 6))
        plt.imshow(image.numpy().transpose((1, 2, 0)))
        plt.label = landmark

        plt.show()

In [22]:
import glob
import os

# Получение всех файлов с расширением .txt в папке и её подкаталогах
files = glob.glob(f'{root}/**/*.png', recursive=True)
for path in files:
    print(os.path.basename(os.path.dirname(path)))
display(files)

data_example_3
data_example_3
data_example_3
data_example_3
data_example_3


['/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3/4.png',
 '/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3/2.png',
 '/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3/3.png',
 '/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3/1.png',
 '/Users/user/Documents/programing/Python/CV/8. Rare traffic signs/tests/00_unittest_dataset_input/data_example_3/0.png']

In [13]:
import json
with open('classes.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [14]:
class_to_idx = {class_name: data[class_name]['id'] for class_name in data}

In [15]:
class_to_idx

{'1.1': 0,
 '1.10': 1,
 '1.11.1': 2,
 '1.11.2': 3,
 '1.12.1': 4,
 '1.12.2': 5,
 '1.13': 6,
 '1.14': 7,
 '1.15': 8,
 '1.16': 9,
 '1.17': 10,
 '1.18': 11,
 '1.19': 12,
 '1.2': 13,
 '1.20.1': 14,
 '1.20.2': 15,
 '1.20.3': 16,
 '1.21': 17,
 '1.22': 18,
 '1.23': 19,
 '1.25': 20,
 '1.26': 21,
 '1.27': 22,
 '1.30': 23,
 '1.31': 24,
 '1.33': 25,
 '1.5': 26,
 '1.6': 27,
 '1.7': 28,
 '1.8': 29,
 '2.1': 30,
 '2.2': 31,
 '2.3.1': 32,
 '2.3.2': 33,
 '2.3.3': 34,
 '2.3.4': 35,
 '2.3.5': 36,
 '2.3.6': 37,
 '2.4': 38,
 '2.5': 39,
 '2.6': 40,
 '2.7': 41,
 '3.1': 42,
 '3.10': 43,
 '3.11.n13': 44,
 '3.11.n17': 45,
 '3.11.n20': 46,
 '3.11.n23': 47,
 '3.11.n5': 48,
 '3.11.n8': 49,
 '3.11.n9': 50,
 '3.12.n10': 51,
 '3.12.n3': 52,
 '3.12.n5': 53,
 '3.12.n6': 54,
 '3.13.r': 55,
 '3.13.r2.5': 56,
 '3.13.r2.6': 57,
 '3.13.r3': 58,
 '3.13.r3.3': 59,
 '3.13.r3.5': 60,
 '3.13.r3.7': 61,
 '3.13.r3.9': 62,
 '3.13.r4.0': 63,
 '3.13.r4.1': 64,
 '3.13.r4.2': 65,
 '3.13.r4.3': 66,
 '3.13.r4.5': 67,
 '3.13.r5': 68,
 '3.1

In [31]:
import torchvision
from torch import nn
model = torchvision.models.resnet50()
model2 = nn.Linear(1000, 20)
c = nn.Sequential(model, model2)
print(c[0][1])


TypeError: 'ResNet' object is not subscriptable

In [ ]:
import torch
import numpy as np
import cv2
from albumentations import Compose, Rotate, RandomBrightnessContrast, GaussianBlur, Normalize, Resize, ToFloat
import albumentations.pytorch as A

# Определите константы
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
NETWORK_SIZE = (224, 224)  # Укажите размер сети, например, 224x224

# Определите аугментации и преобразования
augmentations = [
    Rotate(limit=15, p=0.5),
    RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.25),
    GaussianBlur(p=0.3),
]

common_transforms = [
    Resize(*NETWORK_SIZE),
    ToFloat(max_value=255),
    Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD, max_pixel_value=1.0),
    A.transforms.ToTensorV2(),
]

# Определите полные трансформации
MyFitTransform = Compose(augmentations + common_transforms)
MyPredictTransform = Compose(common_transforms)

# Создайте случайный тензор (изображение)
random_image = (np.random.rand(256, 256, 3) * 255).astype(np.uint8)  # 256x256 случайное изображение с каналами RGB

# Примените преобразования
transformed_image = MyFitTransform(image=random_image)['image']

# Проверка типа и формы результирующего тензора
print(f"Transformed image type: {type(transformed_image)}")
print(f"Transformed image shape: {transformed_image.shape}")


In [ ]:
# -*- coding: utf-8 -*-
import csv
import json
import os
import glob
import pickle
import random
import shutil
import typing
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict

import albumentations as A
import lightning as L
import numpy as np
import scipy
from sklearn.metrics import recall_score
import skimage
import skimage.filters
import skimage.io
import skimage.transform
import torch
import torchvision
import tqdm
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier

# !Этих импортов достаточно для решения данного задания

NETWORK_SIZE = (128, 128)
CLASSES_CNT = 205
BATCH_SIZE = 32
SIMPLE_MAX_EPOCHS = 1
SIMPLE_INTERNAL_FEATURES = 1024
NUM_WORKERS = 11
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if DEVICE == torch.device("cuda:0"):
    torch.set_float32_matmul_precision('medium')

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
augmentations = [
    A.Rotate(limit=15, p=0.5),
    #A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.25),
    A.GaussianBlur(p=0.3),
    #A.HorizontalFlip(p=0.5),
    #A.GaussNoise(var_limit=(5.0, 20.0), p=0.3),
    #A.ElasticTransform(alpha=3, sigma=2, p=0.5),
    #A.Perspective(p=0.25)
]

common_transforms = [
    A.Resize(*NETWORK_SIZE),
    A.ToFloat(max_value=255),
    A.Normalize(max_pixel_value=1.0, mean=IMAGENET_MEAN, std=IMAGENET_STD),
    A.pytorch.transforms.ToTensorV2(),
]

MyFitTransform = A.Compose(augmentations + common_transforms)
MyPredictTransform = A.Compose(common_transforms)



class DatasetRTSD(torch.utils.data.Dataset):
    """
    Класс для чтения и хранения датасета.

    :param root_folders: список путей до папок с данными
    :param path_to_classes_json: путь до classes.json
    """

    def __init__(
        self,
        root_folders: typing.List[str],
        path_to_classes_json: str,
    ) -> None:
        super().__init__()
        self.classes, self.class_to_idx = self.get_classes(path_to_classes_json)
        # список пар (путь до картинки, индекс класса)
        
        self.samples = []
        for root in root_folders:
            paths = glob.glob(f'{root}/**/*.png', recursive=True)
            self.samples += [(path, self.class_to_idx[os.path.basename(os.path.dirname(path))]) for path in paths]
        # cловарь из списков картинок для каждого класса, classes_to_samples[индекс класса] = [список чисел-позиций картинок в self.samples]
        tmp = {}
        for class_num in self.classes:
            tmp[self.class_to_idx[class_num]] = []
        for idx, pair in enumerate(self.samples):
            class_num = pair[1]
            tmp[class_num].append(idx)
        self.classes_to_samples = tmp
        # аугментации + нормализация + ToTensorV2
        self.transform = MyFitTransform

    def __getitem__(self, index: int) -> typing.Tuple[torch.Tensor, str, int]:
        """
        Возвращает тройку: тензор с картинкой, путь до файла, номер класса файла (если нет разметки, то "-1").
        """
        img_path, class_num = self.samples[index]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image, img_path, class_num

    @staticmethod
    def get_classes(
        path_to_classes_json,
    ) -> typing.Tuple[typing.List[str], typing.Mapping[str, int]]:
        """
        Считывает из classes.json информацию о классах.

        :param path_to_classes_json: путь до classes.json
        """
        with open(path_to_classes_json, 'r', encoding='utf-8') as file:
            data = json.load(file)
        ### YOUR CODE HERE - словарь, class_to_idx['название класса'] = индекс
        class_to_idx = {class_name: data[class_name]['id'] for class_name in data}
        ### YOUR CODE HERE - массив, classes[индекс] = 'название класса'
        classes = [0] * 205
        count = 0
        for class_name in data:
            count += 1
            classes[data[class_name]['id']] = class_name
        return classes[:count], class_to_idx

    def __len__(self) -> int:
        """
        Возвращает размер датасета (количество сэмплов).
        """
        return len(self.samples)

class TestData(torch.utils.data.Dataset):
    """
    Класс для чтения и хранения тестового датасета.

    :param root: путь до папки с картинками знаков
    :param path_to_classes_json: путь до classes.json
    :param annotations_file: путь до .csv-файла с аннотациями (опциональный)
    """

    def __init__(
        self,
        root: str,
        path_to_classes_json: str,
        annotations_file: str = None,
    ) -> None:
        super().__init__()
        self.root = root
        self.classes, self.class_to_idx = self.get_classes(path_to_classes_json)
        # список путей до картинок
        self.samples = [os.path.basename(path) for path in glob.glob(f'{root}/**/*.png', recursive=True)]
        # преобразования: ресайз + нормализация + ToTensorV2
        self.transform = MyPredictTransform
        self.targets = None
        if annotations_file is not None:
            # словарь, targets[путь до картинки] = индекс класса
            tmp = {}
            with open(annotations_file, mode='r', encoding='utf-8') as file:
                csv_reader = csv.reader(file)
                next(csv_reader)
                
                for row in csv_reader:
                    img_name = row[0] 
                    class_name = row[1]
                    tmp[img_name] = self.class_to_idx[class_name]
            self.targets = tmp

    def __getitem__(self, index: int) -> typing.Tuple[torch.Tensor, str, int]:
        """
        Возвращает тройку: тензор с картинкой, путь до файла, номер класса файла (если нет разметки, то "-1").
        """
        img_name = self.samples[index]
        image = Image.open(os.path.join(self.root, img_name)).convert("RGB")
        if self.transform:
            image = self.transform(image=np.array(image))
        return image['image'], img_name, self.targets[img_name] if self.targets else -1

    def __len__(self) -> int:
        """
        Возвращает размер датасета (количество сэмплов).
        """
        return len(self.samples)
        
    @staticmethod
    def get_classes(
        path_to_classes_json,
    ) -> typing.Tuple[typing.List[str], typing.Mapping[str, int]]:
        """
        Считывает из classes.json информацию о классах.

        :param path_to_classes_json: путь до classes.json
        """
        with open(path_to_classes_json, 'r', encoding='utf-8') as file:
            data = json.load(file)
        ### YOUR CODE HERE - словарь, class_to_idx['название класса'] = индекс
        class_to_idx = {class_name: data[class_name]['id'] for class_name in data}
        ### YOUR CODE HERE - массив, classes[индекс] = 'название класса'
        classes = [0] * 205
        count = 0
        for class_name in data:
            count += 1
            classes[data[class_name]['id']] = class_name
        return classes[:count], class_to_idx


class CustomNetwork(L.LightningModule):
    """
    Класс, реализующий нейросеть для классификации.

    :param features_criterion: loss-функция на признаки, извлекаемые нейросетью перед классификацией (None когда нет такого лосса)
    :param internal_features: внутреннее число признаков
    """

    def __init__(
        self,
        features_criterion: (
            typing.Callable[[torch.Tensor, torch.Tensor], torch.Tensor] | None
        ) = None,
        internal_features: int = SIMPLE_INTERNAL_FEATURES,
        need_weights=False # ПОМЕНЯТЬ ПЕРЕД ЗАСЫЛКОЙ НА FALSE
    ):
        super().__init__()
        self.features_criterion = features_criterion
        self.model, self.classifier = self.get_model(internal_features, need_weights)
        '''if not need_weights:
            self.load_model()
        '''
    def get_model(self, internal_features, need_weights):
        if need_weights:
            model = torchvision.models.resnet50(weights="IMAGENET1K_V1")
        else:
            model = torchvision.models.resnet50()
        num_features = model.fc.in_features
        model.fc = torch.nn.Linear(num_features, internal_features)
        
        classifier = torch.nn.Sequential(
            torch.nn.ReLU(),
            torch.nn.Linear(internal_features, CLASSES_CNT),
        )

        '''for param in model.parameters():
            param.requires_grad = False
        
        for param in model.fc.parameters():
            param.requires_grad = True
            
        for param in classifier.parameters():
            param.requires_grad = True'''
            
        return model, classifier
    
    def save_model(self):
        torch.save(self.state_dict(), 'simple_model.pth')
        #full_model = torch.nn.Sequential(torch.nn.ModuleList([self.model, self.classifier]))
        #torch.save(full_model.state_dict(), 'simple_model.pth')

    def load_model(self, path_to_model='simple_model.pth'):
        self.load_state_dict(torch.load(path_to_model, map_location="cpu", weights_only=True))
        
    '''def load_model(self, path_to_model='simple_model.pth'):
        full_model = torch.nn.Sequential(torch.nn.ModuleList([self.model, self.classifier]))
        full_model.load_state_dict(torch.load(path_to_model, map_location="cpu", weights_only=True))
        self.model = full_model[0][0]
        self.classifier = full_model[0][1]'''

    def forward(self, x: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        """
        Функция для прогона данных через нейронную сеть.
        Возвращает два тензора: внутреннее представление и логиты после слоя-классификатора.
        """
        features = self.model(x)
        logits = self.classifier(features)
        return features, logits

    def predict(self, x: torch.Tensor) -> np.ndarray:
        """
        Функция для предсказания классов-ответов. Возвращает np-массив с индексами классов.

        :param x: батч с картинками
        """
        self.eval()
        with torch.no_grad():
            _, logits = self(x)
            predictions = torch.argmax(logits, dim=1)
        return predictions.cpu().numpy()

    def print_batch(batch):
        for image, _, landmark in batch:
        image = (image - image.min()) / (image.max() - image.min())

        plt.figure(figsize=(6, 6))
        plt.imshow(image.numpy().transpose((1, 2, 0)))
        plt.label = landmark

        plt.show()

    def training_step(self, batch, batch_idx):
        """
        Один шаг обучения, который включает расчет лосса.
        """
        x, _, y = batch

        features, logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        
        if self.features_criterion is not None:
            features_loss = self.features_criterion(features, y)
            loss += features_loss

        self.running_train_loss += loss.item()
        self.train_batch_count += 1

        running_avg_loss = self.running_train_loss / self.train_batch_count
        self.log("running_train_loss", running_avg_loss, on_step=True, prog_bar=True)

        self.log("train_loss", loss)
        return loss

    def on_train_epoch_start(self):
        """
        Сброс значений накопленного лосса и количества батчей в начале каждой эпохи.
        """
        self.running_train_loss = 0.0
        self.train_batch_count = 0

    def validation_step(self, batch):
        """
        Шаг валидации. Вычисляет и логирует лосс и метрики.
        """
        x, _, y = batch
        features, logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        self.log("val_loss", loss, prog_bar=True)
        
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_acc", acc, prog_bar=True)
        
        return loss


    '''def training_step(self, batch):
        """
        Один шаг обучения, который включает расчет лосса.
        """
        x, _, y = batch
        features, logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        
        if self.features_criterion is not None:
            features_loss = self.features_criterion(features, y)
            loss += features_loss
        
        self.log("train_loss", loss)
        return loss
    
    '''

    def configure_optimizers(self):
        """
        Настройка оптимизаторов.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


def train_simple_classifier() -> torch.nn.Module:
    """
    Функция для обучения простого классификатора с валидацией.
    """
    full_dataset = DatasetRTSD(
        root_folders=["./cropped-train/"],
        path_to_classes_json="./classes.json"
    )

    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model = CustomNetwork()

    trainer = L.Trainer(
        max_epochs=SIMPLE_MAX_EPOCHS,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )
    
    trainer.fit(model, train_loader, val_loader)
    model.save_model()
    return model



def apply_classifier(
    model: CustomNetwork,
    test_folder: str,
    path_to_classes_json: str,
) -> typing.List[typing.Mapping[str, typing.Any]]:
    """
    Функция, которая применяет модель и получает её предсказания.

    :param model: модель, которую нужно протестировать
    :param test_folder: путь до папки с тестовыми данными
    :param path_to_classes_json: путь до файла с информацией о классах classes.json
    """
    test_dataset = TestData(
        root=test_folder,
        path_to_classes_json=path_to_classes_json,
    )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model.eval()
    results = []

    with torch.no_grad():
        for images, img_names, annotations in test_loader:
            #images = images.to(DEVICE)
            predictions = model.predict(images)
            
            for img_name, class_idx, annotation in zip(img_names, predictions, annotations):
                class_name = test_dataset.classes[class_idx]
                results.append({"filename": img_name, "class": class_name, "gt": annotation})
    return results

def calc_metric(y_true, y_pred):
    ok_cnt = 0
    all_cnt = 0
    for t, p in zip(y_true, y_pred):
        all_cnt += 1
        if t == p:
            ok_cnt += 1
    return ok_cnt / max(1, all_cnt)

def test_classifier(
    model: CustomNetwork,
    test_folder: str = "./smalltest/",
    annotations_file: str = "./smalltest_annotations.csv",
    path_to_classes_json:str = "./classes.json",
) -> typing.Tuple[float, float, float]:
    """
    Функция для тестирования качества модели.
    Возвращает точность на всех знаках, Recall на редких знаках и Recall на частых знаках.

    :param model: модель, которую нужно протестировать
    :param test_folder: путь до папки с тестовыми данными
    :param annotations_file: путь до .csv-файла с аннотациями
    """
    # Используем apply_classifier для получения предсказаний
    predictions = apply_classifier(model, test_folder, path_to_classes_json)
    with open(path_to_classes_json, 'r', encoding='utf-8') as file:
        data = json.load(file)
            
    targets = {}
    with open(annotations_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for row in csv_reader:
            img_name = row[0]
            class_name = row[1]
            targets[img_name] = class_name

    all_preds = []
    all_trues = []
    all_types = []
    for pred in predictions:
        img_name = pred['filename']
        predicted_class = pred['class']
        #gt = pred['gt']
        if img_name in targets:
            true_class = targets[img_name]
            all_preds.append(predicted_class)
            all_trues.append(true_class)
            all_types.append(data[true_class]['type'])

    y_pred_rare = []
    y_pred_freq = []
    y_true_rare = []
    y_true_freq = []
    for i in range(len(all_types)):
        if all_types[i] == 'freq':
            y_pred_freq.append(all_preds[i])
            y_true_freq.append(all_trues[i])
        else:
            y_pred_rare.append(all_preds[i])
            y_true_rare.append(all_trues[i])

    total_metric = calc_metric(all_trues, all_preds)
    rare_metric = calc_metric(y_true_rare, y_pred_rare)
    freq_metric = calc_metric(y_true_freq, y_pred_freq)

    return total_metric, rare_metric, freq_metric



class SignGenerator(object):
    """
    Класс для генерации синтетических данных.

    :param background_path: путь до папки с изображениями фона
    """

    def __init__(self, background_path: str) -> None:
        super().__init__()
        ### YOUR CODE HERE

    ### Для каждого из необходимых преобразований над иконками/картинками,
    ### напишите вспомогательную функцию приблизительно следующего вида:
    ###
    ### @staticmethod
    ### def discombobulate_icon(icon: np.ndarray) -> np.ndarray:
    ###     ### YOUR CODE HERE
    ###     return ...
    ###
    ### Постарайтесь не использовать готовые библиотечные функции для
    ### аугментаций и преобразования картинок, а реализовать их
    ### "из первых принципов" на numpy

    def get_sample(self, icon: np.ndarray) -> np.ndarray:
        """
        Функция, встраивающая иконку на случайное изображение фона.

        :param icon: Массив с изображением иконки
        """
        ### YOUR CODE HERE
        icon = ...
        ### YOUR CODE HERE - случайное изображение фона
        bg = ...
        return  ### YOUR CODE HERE


def generate_one_icon(args: typing.Tuple[str, str, str, int]) -> None:
    """
    Функция, генерирующая синтетические данные для одного класса.

    :param args: Это список параметров: [путь до файла с иконкой, путь до выходной папки, путь до папки с фонами, число примеров каждого класса]
    """
    ### YOUR CODE HERE


def generate_all_data(
    output_folder: str,
    icons_path: str,
    background_path: str,
    samples_per_class: int = 1000,
) -> None:
    """
    Функция, генерирующая синтетические данные.
    Эта функция запускает пул параллельно работающих процессов, каждый из которых будет генерировать иконку своего типа.
    Это необходимо, так как процесс генерации очень долгий.
    Каждый процесс работает в функции generate_one_icon.

    :param output_folder: Путь до выходной директории
    :param icons_path: Путь до директории с иконками
    :param background_path: Путь до директории с картинками фона
    :param samples_per_class: Количество примеров каждого класса, которые надо сгенерировать
    """
    shutil.rmtree(output_folder, ignore_errors=True)
    with ProcessPoolExecutor(8) as executor:
        params = [
            [
                os.path.join(icons_path, icon_file),
                output_folder,
                background_path,
                samples_per_class,
            ]
            for icon_file in os.listdir(icons_path)
        ]
        list(tqdm.tqdm(executor.map(generate_one_icon, params)))


def train_synt_classifier() -> torch.nn.Module:
    """
    Функция для обучения простого классификатора на смеси исходных и ситетических данных.
    """
    ### YOUR CODE HERE
    return model


class FeaturesLoss(torch.nn.Module):
    """
    Класс для вычисления loss-функции на признаки предпоследнего слоя нейросети.
    """

    def __init__(self, margin: float) -> None:
        super().__init__()
        ### YOUR CODE HERE

    def forward(self, outputs: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Функция, вычисляющая loss-функцию на признаки предпоследнего слоя нейросети.

        :param outputs: Признаки с предпоследнего слоя нейросети
        :param labels: Реальные метки объектов
        """
        ### YOUR CODE HERE


class CustomBatchSampler(torch.utils.data.sampler.Sampler[typing.List[int]]):
    """
    Класс для семплирования батчей с контролируемым числом классов и примеров каждого класса.

    :param data_source: Это датасет RTSD
    :param elems_per_class: Число элементов каждого класса
    :param classes_per_batch: Количество различных классов в одном батче
    """

    def __init__(
        self,
        data_source: DatasetRTSD,
        elems_per_class: int,
        classes_per_batch: int,
    ) -> None:
        ### YOUR CODE HERE
        pass

    def __iter__(self):
        """
        Функция, которая будет генерировать список индексов элементов в батче.
        """
        ### YOUR CODE HERE

    def __len__(self) -> None:
        """
        Возвращает общее количество батчей.
        """
        ### YOUR CODE HERE


def train_better_model() -> torch.nn.Module:
    """
    Функция для обучения классификатора на смеси исходных и ситетических данных с новым лоссом на признаки.
    """
    ### YOUR CODE HERE
    return model


class ModelWithHead(CustomNetwork):
    """
    Класс, реализующий модель с головой из kNN.

    :param n_neighbors: Количество соседей в методе ближайших соседей
    """

    def __init__(self, n_neighbors: int) -> None:
        super().__init__()
        self.eval()
        ### YOUR CODE HERE

    def load_nn(self, nn_weights_path: str) -> None:
        """
        Функция, загружающая веса обученной нейросети.

        :param nn_weights_path: Это путь до весов обученной нейросети с улучшенными признаками на предпоследнем слое
        """
        ### YOUR CODE HERE

    def load_head(self, knn_path: str) -> None:
        """
        Функция, загружающая веса kNN (с помощью pickle).

        :param knn_path: Путь, откуда надо прочитать веса kNN
        """
        ### YOUR CODE HERE

    def save_head(self, knn_path: str) -> None:
        """
        Функция, сохраняющая веса kNN (с помощью pickle).

        :param knn_path: Путь, куда надо сохранить веса kNN
        """
        ### YOUR CODE HERE

    def train_head(self, indexloader: torch.utils.data.DataLoader) -> None:
        """
        Функция, обучающая голову kNN.

        :param indexloader: Загрузчик данных для обучения kNN
        """
        ### YOUR CODE HERE

    def predict(self, imgs: torch.Tensor) -> np.ndarray:
        """
        Функция для предсказания классов-ответов. Возвращает np-массив с индексами классов.

        :param imgs: батч с картинками
        """
        ### YOUR CODE HERE - предсказание нейросетевой модели
        features, model_pred = ...
        features = features / np.linalg.norm(features, axis=1)[:, None]
        ### YOUR CODE HERE - предсказание kNN на features
        knn_pred = ...
        return knn_pred


class IndexSampler(torch.utils.data.sampler.Sampler[int]):
    """
    Класс для семплирования батчей с картинками индекса.

    :param data_source: Это датасет RTSD с синтетическими примерами
    :param examples_per_class: Число элементов каждого класса, которые должны попасть в индекс
    """

    def __init__(self, data_source: DatasetRTSD, examples_per_class: int) -> None:
        ### YOUR CODE HERE
        pass

    def __iter__(self):
        """
        Функция, которая будет генерировать список индексов элементов в батче.
        """
        return  ### YOUR CODE HERE

    def __len__(self) -> int:
        """
        Возвращает общее количество индексов.
        """
        ### YOUR CODE HERE


def train_head(nn_weights_path: str, examples_per_class: int = 20) -> torch.nn.Module:
    """
    Функция для обучения kNN-головы классификатора.

    :param nn_weights_path: Это путь до весов обученной нейросети с улучшенными признаками на предпоследнем слое
    :param examples_per_class: Число элементов каждого класса, которые должны попасть в индекс
    """
    ### YOUR CODE HERE


if __name__ == "__main__":
    # The following code won't be run in the test system, but you can run it
    # on your local computer with `python -m rare_traffic_sign_solution`.

    # Feel free to put here any code that you used while
    # debugging, training and testing your solution.
    pass
    train_simple_classifier()
